***
***
## 0. Setup

In [99]:
# Load the autoreload extension
%load_ext autoreload

# Enable autoreloading for all modules
%autoreload 2

# Import local modules
from util_core import *
import olympic_medals.util_olympic_medals as U


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


***
***
## 1. Data load

In [100]:
# Base loads
all_data = U.load_base_datasets()

medal_df = all_data["medal_df"]
medalist_df = all_data["medalist_df"]
event_df = all_data["event_df"]
athlete_df = all_data["athlete_df"]
team_df = all_data["team_df"]

# Combined entry data
entry_df = U.build_combined_event_entries(team_df, athlete_df)
all_data["entry_df"] = entry_df

# Store
save_object(all_data, "in/olympic_medals_all_data.p")


[medal] Dropped 0/1044 (0.0%) records with null event details.
[medal] Dropped 0/2315 (0.0%) records with null event details.
[event] Dropped 0/329 (0.0%) records with null event details.
[team] Dropped 358/1698 (21.1%) non-current records.
[team] Dropped 0/1340 (0.0%) records with null event details.
[team] Added medals for 284/1340 (21.2%) records.
[athlete] Dropped 3/11113 (0.0%) non-current records.
[athlete] Dropped 0/11110 (0.0%) records with null event details.
[athlete] Exploded to 14978 records, one per athlete per event entered.
[athlete] Dropped 435/14978 (2.9%) invalid event records.
[athlete] Added medals for 2298/14543 (15.8%) records.
[combining]: Removed 6494/14543 (44.7%) athlete records for Team type events.


#### 1.1 Medals

In [101]:
# Side quest - how many medals were awarded in each event?
count_df = medal_df["event_key"].value_counts().to_frame().reset_index()

# Print summary stats
total_e = len(count_df)
gt3 = len(count_df[count_df["count"]>3])
eq3 = len(count_df[count_df["count"]==3])
lt3 = len(count_df[count_df["count"]<3])
print(f"Isolated a total of {total_e} events awarding {len(medal_df)} cummulative medals...")
print(f"{gt3} ({gt3/total_e*100:.1f}%) events awarded 4+ medals")
print(f"{eq3} ({eq3/total_e*100:.1f}%) events awarded 3 medals")
print(f"{lt3} ({lt3/total_e*100:.1f}%) events awarded <3 medals")

# Look at 57 events with 4+ medals
gt3_df = count_df[count_df["count"]>3].copy()
gt3_df["discipline"] = gt3_df["event_key"].apply(lambda x: x.split(": ")[0])
print(gt3_df["discipline"].value_counts())

# View sample
medal_df.head()


Isolated a total of 329 events awarding 1044 cummulative medals...
57 (17.3%) events awarded 4+ medals
272 (82.7%) events awarded 3 medals
0 (0.0%) events awarded <3 medals
discipline
wrestling              18
judo                   15
boxing                 13
taekwondo               8
athletics               1
canoe sprint            1
artistic gymnastics     1
Name: count, dtype: int64


,medal_type,medal_code,medal_date,name,gender,discipline,event,event_type,url_event,code,country_code,country,country_long,event_key
0,Gold Medal,1.0,2024-07-27,Remco EVENEPOEL,M,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1903136,BEL,Belgium,Belgium,cycling road: men's individual time trial
1,Silver Medal,2.0,2024-07-27,Filippo GANNA,M,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1923520,ITA,Italy,Italy,cycling road: men's individual time trial
2,Bronze Medal,3.0,2024-07-27,Wout van AERT,M,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1903147,BEL,Belgium,Belgium,cycling road: men's individual time trial
3,Gold Medal,1.0,2024-07-27,Grace BROWN,W,Cycling Road,Women's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/women-s-in...,1940173,AUS,Australia,Australia,cycling road: women's individual time trial
4,Silver Medal,2.0,2024-07-27,Anna HENDERSON,W,Cycling Road,Women's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/women-s-in...,1912525,GBR,Great Britain,Great Britain,cycling road: women's individual time trial


#### 1.2 Events

In [102]:
# Print summary stats and view sample
total_e2 = len(event_df)
assert total_e2 == total_e, f"Warning total_e: {total_e}, total_e2: {total_e2}"
print(f"Isolated a total of {total_e2} events...")
event_df.head()


Isolated a total of 329 events...


,event,tag,sport,sport_code,sport_url,event_key
0,Men's Individual,archery,Archery,ARC,https://olympics.com/en/paris-2024/sports/archery,archery: men's individual
1,Women's Individual,archery,Archery,ARC,https://olympics.com/en/paris-2024/sports/archery,archery: women's individual
2,Men's Team,archery,Archery,ARC,https://olympics.com/en/paris-2024/sports/archery,archery: men's team
3,Women's Team,archery,Archery,ARC,https://olympics.com/en/paris-2024/sports/archery,archery: women's team
4,Mixed Team,archery,Archery,ARC,https://olympics.com/en/paris-2024/sports/archery,archery: mixed team


#### 1.3 Teams

In [103]:
# View sample
team_df.head()


,code,current,team,team_gender,country_code,country,country_long,discipline,disciplines_code,event,athletes,coaches,athletes_codes,num_athletes,coaches_codes,num_coaches,event_key,medal_flag,medal_type
0,arcmteam3---chn01,True,People's Republic of China,M,CHN,China,People's Republic of China,archery,ARC,men's team,"['KAO Wenchao', 'LI Zhongyuan', 'WANG Yan']",NaN,"['1913366', '1913367', '1913369']",3.0,NaN,NaN,archery: men's team,False,NaN
1,arcmteam3---col01,True,Colombia,M,COL,Colombia,Colombia,archery,ARC,men's team,"['ARCILA Santiago', 'ENRIQUEZ Jorge', 'HERNAND...",NaN,"['1935642', '1543412', '1935644']",3.0,NaN,NaN,archery: men's team,False,NaN
2,arcmteam3---fra01,True,France,M,FRA,France,France,archery,ARC,men's team,"['ADDIS Baptiste', 'CHIRAULT Thomas', 'VALLADO...",NaN,"['1541270', '1541272', '1541275']",3.0,NaN,NaN,archery: men's team,True,Silver Medal
3,arcmteam3---gbr01,True,Great Britain,M,GBR,Great Britain,Great Britain,archery,ARC,men's team,"['HALL Conor', 'HALL Tom', 'WISE Alex']",NaN,"['1560988', '1560989', '1561003']",3.0,NaN,NaN,archery: men's team,False,NaN
4,arcmteam3---ind01,True,India,M,IND,India,India,archery,ARC,men's team,"['BOMMADEVARA Dhiraj', 'JADHAV Pravin Ramesh',...",NaN,"['1546108', '1546112', '1546110']",3.0,NaN,NaN,archery: men's team,False,NaN


#### 1.4 Athletes

In [104]:
# View sample
athlete_df.head()


,code,current,name,name_short,name_tv,gender,function,country_code,country,country_long,...,philosophy,sporting_relatives,ritual,other_sports,event_key,discipline,event,event_type,medal_flag,medal_type
0,1532872,True,ALEKSANYAN Artur,ALEKSANYAN A,Artur ALEKSANYAN,Male,Athlete,ARM,Armenia,Armenia,...,"""Wrestling is my life."" (mediamax.am. 18 May 2...",NaN,NaN,NaN,wrestling: men's greco-roman 97kg,wrestling,men's greco-roman 97kg,Individual,True,Silver Medal
1,1532873,True,AMOYAN Malkhas,AMOYAN M,Malkhas AMOYAN,Male,Athlete,ARM,Armenia,Armenia,...,"""To become a good athlete, you first have to b...","Uncle, Roman Amoyan (wrestling), 2008 Olympic ...",NaN,NaN,wrestling: men's greco-roman 77kg,wrestling,men's greco-roman 77kg,Individual,True,Bronze Medal
2,1532874,True,GALSTYAN Slavik,GALSTYAN S,Slavik GALSTYAN,Male,Athlete,ARM,Armenia,Armenia,...,NaN,NaN,NaN,NaN,wrestling: men's greco-roman 67kg,wrestling,men's greco-roman 67kg,Individual,False,NaN
3,1532944,True,HARUTYUNYAN Arsen,HARUTYUNYAN A,Arsen HARUTYUNYAN,Male,Athlete,ARM,Armenia,Armenia,...,"“Nothing is impossible, set goals in front of ...",NaN,NaN,NaN,wrestling: men's freestyle 57kg,wrestling,men's freestyle 57kg,Individual,False,NaN
4,1532945,True,TEVANYAN Vazgen,TEVANYAN V,Vazgen TEVANYAN,Male,Athlete,ARM,Armenia,Armenia,...,NaN,NaN,NaN,NaN,wrestling: men's freestyle 65kg,wrestling,men's freestyle 65kg,Individual,False,NaN


#### 1.5 Combined entries

In [105]:
# View sample
entry_df.head()


,event_key,discipline,event,gender,country_code,country,num_athletes,medal_flag,medal_type
0,archery: men's team,archery,men's team,M,CHN,China,3.0,False,NaN
1,archery: men's team,archery,men's team,M,COL,Colombia,3.0,False,NaN
2,archery: men's team,archery,men's team,M,FRA,France,3.0,True,Silver Medal
3,archery: men's team,archery,men's team,M,GBR,Great Britain,3.0,False,NaN
4,archery: men's team,archery,men's team,M,IND,India,3.0,False,NaN


***
***
## 2. Medal rates

In [106]:
# Load
all_data = load_object("in/olympic_medals_all_data.p")

entry_df = all_data["entry_df"]


In [107]:
# Build medal rate df
rate_df = U.calc_country_medal_rates(entry_df)

# Export and view sample
rate_df.to_csv(f"out/country_medal_rankings.csv", index=False)
rate_df


,country,entries,total_medals,gold_medals,silver_medals,bronze_medals,total_medals_weighted,medal_rate_unweighted,medal_rate_weighted,medal_rate_unweighted_rank,medal_rate_weighted_rank
0,Saint Lucia,5,2,1,1,0,3.5,0.4,0.700000,1.0,1.0
1,Dominica,4,1,1,0,0,2.0,0.25,0.500000,9.0,2.0
2,DPR Korea,15,6,0,2,4,7.0,0.4,0.466667,1.0,3.0
3,IR Iran,41,12,3,6,3,18.0,0.292683,0.439024,5.0,4.0
4,Bahrain,15,4,2,1,1,6.5,0.266667,0.433333,6.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...
201,Vietnam,19,0,0,0,0,0.0,0.0,0.000000,93.0,93.0
202,"Virgin Islands, B",4,0,0,0,0,0.0,0.0,0.000000,93.0,93.0
203,"Virgin Islands, US",5,0,0,0,0,0.0,0.0,0.000000,93.0,93.0
204,Yemen,4,0,0,0,0,0.0,0.0,0.000000,93.0,93.0


***
***
## 3. Plots

#### 3.1 Medal Wins vs Chances

In [108]:
# Setup
config = {
    # Data
    "name_col": "country",
    "size_col": "entries",
    "color_col": "entries",
    "x_axis_col": "entries",
    "y_axis_col": "total_medals",
    # Labels
    "title": "Medal Wins vs Chances",
    "x_axis_title": "Medal Chances",
    "y_axis_title": "Medal Wins",
    # Formatting
    "min_point_size": 5,
    "max_point_size": 30,
    "x_axis_dtick": 20,
    "y_axis_dtick": 10,
    "width": 800,
    "height": 800
}

# Plot and save html version
fig = plotly_scatter_v1(rate_df, config)
fig.write_html("md/olympic_medals/medal_wins_vs_chances.html")


#### 3.2 Weighted Medal Wins vs Chances

In [109]:
# Setup
config = {
    # Data
    "name_col": "country",
    "size_col": "entries",
    "color_col": "entries",
    "x_axis_col": "entries",
    "y_axis_col": "total_medals_weighted",
    # Labels
    "title": "Weighted Medal Wins vs Chances",
    "x_axis_title": "Medal Chances",
    "y_axis_title": "Weighted Medal Wins (Gold = 2, Silver = 1.5, Bronze = 1)",
    # Formatting
    "min_point_size": 5,
    "max_point_size": 30,
    "x_axis_dtick": 20,
    "y_axis_dtick": 10,
    "width": 800,
    "height": 800
}

# Plot and save html version
fig = plotly_scatter_v1(rate_df, config)
fig.write_html("md/olympic_medals/weighted_medal_wins_vs_chances.html")


#### 3.3 Medal Rate vs Chances

In [110]:
# Setup
config = {
    # Data
    "name_col": "country",
    "size_col": "entries",
    "color_col": "entries",
    "x_axis_col": "entries",
    "y_axis_col": "medal_rate_unweighted",
    # Labels
    "title": "Medal Rate vs Chances",
    "x_axis_title": "Medal Chances",
    "y_axis_title": "Medal Rate",
    # Formatting
    "min_point_size": 5,
    "max_point_size": 30,
    "x_axis_dtick": 20,
    "y_axis_dtick": 0.1,
    "width": 800,
    "height": 800
}

# Plot and save html version
fig = plotly_scatter_v1(rate_df, config)
fig.write_html("md/olympic_medals/medal_rate_vs_chances.html")


#### 3.4 Weighted Medal Rate vs Chances

In [111]:
# Setup
config = {
    # Data
    "name_col": "country",
    "size_col": "entries",
    "color_col": "entries",
    "x_axis_col": "entries",
    "y_axis_col": "medal_rate_weighted",
    # Labels
    "title": "Weighted Medal Rate vs Chances",
    "x_axis_title": "Medal Chances",
    "y_axis_title": "Weighted Medal Rate (Gold = 2, Silver = 1.5, Bronze = 1)",
    # Formatting
    "min_point_size": 5,
    "max_point_size": 30,
    "x_axis_dtick": 20,
    "y_axis_dtick": 0.1,
    "width": 800,
    "height": 800
}

# Plot and save html version
fig = plotly_scatter_v1(rate_df, config)
fig.write_html("md/olympic_medals/weighted_medal_rate_vs_chances.html")


***
***
## 3. Other analysis

In [112]:
rate_df[["entries", "total_medals"]].corr()


,entries,total_medals
entries,1.00000,0.89764
total_medals,0.89764,1.00000


In [113]:
rate_df[["entries", "medal_rate_unweighted", "medal_rate_weighted"]].corr()


,entries,medal_rate_unweighted,medal_rate_weighted
entries,1.000000,0.372375,0.392037
medal_rate_unweighted,0.372375,1.000000,0.976465
medal_rate_weighted,0.392037,0.976465,1.000000


In [ ]:
# To do...

# Athletes
# How many total athletes
# How many competing only individually, only on team, both
# How many athletes earn 1+ medal?
# How many earn 2+, 3+, 4+?
# Do your chances of earning a medal increase if you play a team sport?
# Which athletes have the most chances at medals (event entries), and what sports do they play

# Teams
# How many total teams
# Largest team size
# How many teams earn 1+ medal
# How many earn 2+, 3+, 4+?

# Combined
# Team size (count of unique athletes) vs event entries (chances at a medal)
# Which countries do a better job of maximizing their medal chances (ratio of team size to event entries is smaller)
